# Module 4 Backpropagation

This is how our network performs gradient descent. By treating the network as a chain of functions we can use the chain rule to find the gradients for every single weight. Using the gradients, we can update the weights and biases and make the network perform slightly better. We repeat this process until the network stops improving or after a fixed number of iterations. 

So what is our actual chain of operations that we will have our network perform? The first operation is to multiply the weights by our input and add the biases. The second is the activation function which takes in the output of the previous operation. The third is the cost function which takes in the output of the activation function. This would be the exact steps if we had a network with only one layer. If we use more layers we just repeat the first two steps for each layer.


\begin{align}
& \quad x = input \\
\mathrm{(1)} & \quad z(x) = wx + b  \\
\mathrm{(2)} & \quad a(z) = \text{we could use sigmoid, relu, linear, or any other activation} \\
\mathrm{(3)} & \quad C(a) = \frac{1}{2}\|y - a\|^2
\end{align}


We ultimately want to lower the cost or make the answers the network gives us closer to their actual value which we get from our labeled data. Therefore, we want to find the gradient of the cost function and use that to get the gradients of our weights and biases with respect to the cost. In other words we will find out what effect each weight and bias has on the cost function. This effect represents the slope of the cost function in the dimension of a particular weight or bias. That slope is what we want to "roll" down by subtracting it from the particular weight.

How can we compute the gradient for the weights in our network? It is similar to how we found the gradients for the slope and y intercept of our best fit line in the previous module except we have one extra step which is the activation function.

The gradients for the weights and biases in our weighted input function, number 1 above, is the same as with our best fit line. The gradient for the cost function is the same as well. The only difference is now we need to know the gradient of the activation function. However, once we have that, we can find the gradients for all the weights and biases.

We find the gradients for every step in our chain

\begin{align}
x & = \text{input} \\
y & = \text{input label or "correct answer"} \\
\frac{\partial z}{\partial w} & = x \\
\frac{\partial z}{\partial b} & = 1 \\
\frac{\partial a}{\partial z} & = a'(z) \\
\frac{\partial C}{\partial a} & = a - y
\end{align}

For each of our activation functions we can find the derivative or gradient in the following way. The functions listed are linear, relu, and sigmoid respectively.

\begin{align}
y(z) & = z \\
y'(z) & = 1 \\
relu(z) & = \begin{cases}
    z & \quad \text{if } z \text{ > 0} \\
    0 & \quad \text{if } z \text{ <= 0}
    \end{cases} \\
relu'(z) & = 
    \begin{cases}
    1 & \quad \text{if } z \text{ > 0} \\
    0 & \quad \text{if } z \text{ <= 0}
    \end{cases} \\
\sigma(z) & = \frac{1}{1+e^{-z}} \\
\sigma'(z) & = \sigma(z)(1 - \sigma(z))
\end{align}


We can chain the gradients together to find what we really want which is the gradients for each weight and bias in the cost function. In other words, how much does a change in the weight or bias affect the final cost and in what direction (positive or negative). Using this value, we can subtract it from the weight or bias to make it "roll" down the hill. 

So the expression $\frac{\partial C}{\partial w}$ really means what is the change in cost (top) divided by the change in weight (bottom). Or $\frac{\partial C}{\partial b}$ means what is the change in final cost over the change in a certain bias. These values tell us how much a change in the weight or bias affects the final cost and whether that relationship is positive or negative. 

If the relationship is positive that means we have a positive slope and as our weight or bias increases, so too does our cost. Alternatively, if the relationship is negative then as one goes up the other goes down. The magnitude of this value represents how big of a change will happen or how steep our slope is. In that sense it tells us the steepness and direction of the hill. 

So when we chain the gradients together for each of our steps, we can find those slopes.

\begin{align}
\frac{\partial C}{\partial w} & =  \frac{\partial C}{\partial a} \frac{\partial a}{\partial z} \frac{\partial z}{\partial w} & = x a'(z) (a - y)\\
\frac{\partial C}{\partial b} & = \frac{\partial C}{\partial a} \frac{\partial a}{\partial z} \frac{\partial z}{\partial b} & = a'(z) (a - y) \\
\end{align}


Can you see how the tops and bottoms of the fractions in the middle cancel out to give us the left hand side of each equation? All of the a'(z) terms above can be replaced with the corresponding derivative of whichever activation function we decide to use. For instance if we're using the linear activation function, the $a'(z)$ just becomes 1.

Let's see if we can train a single layer network to predict house prices. This layer will only have 1 output for price and we will use a linear activation function. 

## The Data

In [3]:
from sklearn.datasets import load_boston

dataset = load_boston()

# The house features are essentially a table with 13 columns
# each column is described in the dataset
house_features = dataset.data
house_prices = dataset.target

print(dataset.keys())
print(dataset.DESCR)

dict_keys(['data', 'target', 'feature_names', 'DESCR', 'filename'])
.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX

# 1 Layer

To start with this problem of predicting house prices we have to build a network with only 1 layer and only 1 neuron.

![1layer](img/1-layer.svg)

In [5]:
from random import randint
import numpy as np

class Layer:
    def __init__(self, num_inputs, num_neurons):
        # we now have 1 row per neuron and 1 column per weight
        self.weights = np.random.uniform(-.1, .1, size=(num_neurons, num_inputs))

        # we also randomly create a bias for each neuron
        self.biases = np.random.uniform(-.1, .1, size=num_neurons)
    
    def predict(self, inputs):
        return self.weights.dot(inputs) + self.biases

            
    
# create a layer with the same number of inputs as there are features in our data
# and with only 1 output for the price
all_inputs = Layer(len(dataset.feature_names), 1)

# randomly pick a house from the dataset
house_idx = randint(0, len(dataset.data)-1)

# get the features and price of the house
house = house_features[house_idx]
price = house_prices[house_idx]

# have our layer predict a price for the house
predicted_price = all_inputs.predict(house)[0]

print('predicted price: ${:.2f}, actual price: ${:.2f}'.format(predicted_price * 1000, price * 1000))

predicted price: $31978.54, actual price: $12700.00


Obviously the layer is not very good because we have not taught it anything. This is where gradient descent comes in. Now for every example in our data we can find the gradients and use them to update our layer. So we have 4 gradients to find for each input example

\begin{align}
1. \quad \frac{\partial z}{\partial w} & = \text{What effect does a change in our weight have on the output of the weighted input} \\
2. \quad \frac{\partial z}{\partial b} & = \text{What effect does a change in our bias have on the output of the weighted input} \\
3. \quad \frac{\partial a}{\partial z} & = \text{What effect does a change in our weighted input have on the output of the activation function} \\
4. \quad \frac{\partial C}{\partial a} & = \text{What effect does a change in our activation have on the output of the cost function}
\end{align}

We multiply gradients (1, 3, 4) above for the weight gradient and gradients (2, 3, 4) for the biases for every input example. Multiply by the learning rate, and subtract it from the weights or biases. We are using the linear activation function so gradient 3 $\quad \frac{\partial a}{\partial z} = 1$ and we can basically ignore it because anything multiplied by 1 is just itself.

We already saw in the last module that $\frac{\partial z}{\partial w}$ is just $x$ or our input and $\frac{\partial z}{\partial b}$ is just 1. After ignoring all the 1's we are left with only 2 gradients we need to find.

\begin{align}
1. \quad \frac{\partial z}{\partial w} & = x \\
2. \quad \frac{\partial C}{\partial a} & = a - y
\end{align}

Then to find the gradients of our weights and biases we just do



\begin{align}
\text{weights} \quad \frac{\partial C}{\partial w} = \frac{\partial C}{\partial a} \frac{\partial a}{\partial z} \frac{\partial z}{\partial w} = (a-y) * 1 * x = x (a-y) \\
\text{biases} \quad \frac{\partial C}{\partial b} = \frac{\partial C}{\partial a} \frac{\partial a}{\partial z} \frac{\partial z}{\partial b} = (a-y) * 1 * 1 = (a-y)
\end{align}

Where $a$ is the output activation of our layer (i.e. the predicted price), $x$ is just the input (i.e. the house features), and $y$ is the target value (i.e. the correct house price). Numpy also allows us to do a fancy trick where we can take two lists and subtract them which in turn just performs element-wise subtraction. Similarly, if we multiply two numpy arrays, it will just do element-wise multiplication. Therefore, we can find the gradients for all input examples without needing to iterate over them.

In the next example we also introduce input normalization. This takes all input features and rescales them. The normalize function specifically uses an l2 norm. If we then look at a specific house example, we have rescaled all the features of it so that when we use the pythagorean theorum across all features, it equals 1. Since there are 13 features, it would be like $\sqrt{a^2 + b^2 + c^2 + ... + m^2} = 1$

There are a couple reasons for doing this. One is that it makes all the numbers MUCH smaller. That lets us deal with smaller numbers when we update the gradients of the weights and biases which prevents overflow values from accumulating. Also it makes the training much much faster because we no longer have to deal with operations on massive values.

In [9]:
from sklearn.preprocessing import normalize
from numpy.linalg import norm



def cost(y, a):
    return (1/2)*(y - a)**2


def get_predictions(input_data, predictor):
    return [predictor(x)[0] for x in input_data]


# take our 10 examples along with their 10 corresponding 
# predictions and find the cost for each one, then take the average
def find_avg_cost(test_set, test_labels, predictor):
    test_a = get_predictions(test_set, predictor)
    test_costs = [cost(y, a) for (y, a) in zip(test_labels, test_a)]
    return sum(test_costs)/len(test_costs)

def evaluate_predictor(test_set, actual_values, predictor, N=10):
    print('cost: {:.2f}'.format(find_avg_cost(test_set, actual_values, predictor)))
    
    trained_predictions = get_predictions(test_set, predictor)
    
    print('\npredicted\tactual')
    print('---------------------')
    print('\n'.join('{:.2f}\t\t{:.2f}'\
                    .format(a, y) for (a, y) in zip(trained_predictions, actual_values)))

def train(layer, data, labels, learning_rate):
    for i in range(1000):
        a = [layer.predict(x)[0] for x in data]

        # every element in dC/db is just a single value 
        # representing the gradient for the single bias 
        # because we only have 1 neuron. There are 506
        # gradient values, 1 for each example
        dC_db = a - labels

        # every element in dC/dw has 13 values, 1 for 
        # each weight in our layer. There are 506
        # elements, 1 for each example
        dC_dw = (data.transpose() * (a - labels)).transpose()

        # THIS IS THE ACTUAL LEARNING PART!!!
        layer.biases -= dC_db.mean()*learning_rate

        # here the axis 0 means we want to take the 
        # mean of every column so we're left with 13 
        # means and not just one mean of every value 
        # in the list
        layer.weights -= dC_dw.mean(axis=0)*learning_rate

# The all inputs layer is going to use all input columns
# to predict house price
layer = Layer(len(dataset.feature_names), 1)


# try out different learning rates and see how horrible it can get
learning_rate = .6

house_features_normalized = normalize(house_features)

test_start = randint(0, len(house_features_normalized) - 10)

test_set = house_features_normalized[test_start:test_start+10]
actual_values = house_prices[test_start:test_start+10]


print('====== BEFORE TRAINING ======')
evaluate_predictor(test_set, actual_values, layer.predict)

train(layer, house_features_normalized, house_prices, learning_rate)

print('\n\n====== AFTER TRAINING ======')
evaluate_predictor(test_set, actual_values, layer.predict)

====== BEFORE TRAINING ======
cost: 197.75

predicted	actual
---------------------
0.13		20.40
0.13		19.80
0.13		19.40
0.13		21.70
0.12		22.80
0.13		18.80
0.13		18.70
0.12		18.50
0.12		18.30
0.12		21.20


====== AFTER TRAINING ======
cost: 1.99

predicted	actual
---------------------
21.98		20.40
22.04		19.80
21.88		19.40
22.66		21.70
21.65		22.80
21.18		18.80
21.15		18.70
21.45		18.50
20.13		18.30
21.65		21.20


# From 1 To N Layers

IT LEARNS!!! Our cost always gets smaller and our prices are getting closer!!! That means our network is actually getting better at predicting house prices!!!

However, this network only has 1 layer. How might we find the gradients if we have more than 1 layer. We can continue to use the chain rule just as we did before, we just need to add an extra step. This is what our chain of operations looks like when we go from 1 to 2 layers.

![title](img/layers.svg)

We know the gradients for the weighted input $\frac{\partial z}{\partial w}$ and $\frac{\partial z}{\partial b}$, activation functions $\frac{\partial a}{\partial z}$, and cost function $\frac{\partial C}{\partial a}$. Now we just have to find the gradient between the two layers. The input to layer 2 is the output of layer 1. The first operation in layer 2 is the weighted input operation and its input is the activation function output of layer 1.

We want to know how the input to the weighted input function affects it's output. Before our gradients $\frac{\partial z}{\partial w}$ and $\frac{\partial z}{\partial b}$ told us how the weights and biases affect the output, but now we want $\frac{\partial z}{\partial x}$ where x is the output from layer 1. If we replace the variable $x$ with $a$ so that we know our intput represents and activation we get the following gradient for the equation $z(a) = wa + b$

\begin{equation*}
\frac{\partial z}{\partial a} = w
\end{equation*}

In this case $w$ represents our slope and tells us how much a change to $a$ will affect the output of $z(a)$. Now we can simply add this gradient into our chain to find the gradients for the weights and biases of each layer.

Now that we have multiple layers we need to identify which weights and biases we're talking about, the ones from layer 1 or 2. Therefore, from now on when we refer to the weights, biases, or activations we will add a superscript that indicates which layer they are in. The steps to find the gradients for layer one are as follows.

\begin{equation*}
\frac{\partial C}{\partial w^{L1}} = \frac{\partial C}{\partial a^{L2}} \frac{\partial a^{L2}}{\partial z^{L2}} \frac{\partial z^{L2}}{\partial a^{L1}} \frac{\partial a^{L1}}{\partial z^{L1}} \frac{\partial z^{L1}}{\partial w^{L1}}
\end{equation*}

You can see that the fraction or gradient $\frac{\partial z^{L2}}{\partial a^{L1}}$ represents the effect that the output activation of $L1$ has on the output from the weighted input of $L2$. This term is precisely how we can allow our gradients to propagate backwards across multiple layers, hence backpropagation. However, this is really just the chain rule which we're using because we have a chain of operations where the output from one goes on to be the input of the next. The equation for the biases in $L1$ is the same except for the very last term which we replace with $\frac{\partial z^{L1}}{\partial b^{L1}}$ and we've already determined that this term is just 1 so you can ignore it if you like.

Now we just need the gradients for the weights and biases in $L2$ which will look very similar to our network with only one layer. The gradient terms we would like to find are $\frac{\partial C}{\partial w^{L2}}$ and $\frac{\partial C}{\partial b^{L2}}$ and we can do so as follows.

\begin{equation*}
\frac{\partial C}{\partial w^{L2}} = \frac{\partial C}{\partial a^{L2}} \frac{\partial a^{L2}}{\partial z^{L2}} \frac{\partial z^{L2}}{\partial w^{L2}}
\end{equation*}

Just as before, by replacing the laste term $\frac{\partial z^{L2}}{\partial w^{L2}}$ with $\frac{\partial z^{L2}}{\partial b^{L2}}$ we can obtain the gradients for the $L2$ biases. Ultimately, the value of $\frac{\partial z^{L2}}{\partial b^{L2}}$ is just 1 therefore we can essentially ignore it.

# Putting It All Together

We now have all the information we need to find the gradients of the weights and biases for every layer in our network. We have only explicitly mentioned 2 layers, but this process of chaining gradients together can be repeated for any number of layers.

It's killing me!!! what are the final gradient values substituting each respective term in our chain??? If we continue to use the linear activation function, all gradients are just 1. However, if we used another activation function we could replace the 1 with the derivative of whichever activation function we choose.

\begin{align}
\frac{\partial C}{\partial w^{L1}} & = \frac{\partial C}{\partial a^{L2}} \frac{\partial a^{L2}}{\partial z^{L2}} \frac{\partial z^{L2}}{\partial a^{L1}} \frac{\partial a^{L1}}{\partial z^{L1}} \frac{\partial z^{L1}}{\partial w^{L1}}  = (Y - a^{L2}) * 1 * w^{L2} * 1 * x  = (Y - a^{L2}) * w^{L2} * x \\
\frac{\partial C}{\partial b^{L1}} & = \frac{\partial C}{\partial a^{L2}} \frac{\partial a^{L2}}{\partial z^{L2}} \frac{\partial z^{L2}}{\partial a^{L1}} \frac{\partial a^{L1}}{\partial z^{L1}} \frac{\partial z^{L1}}{\partial b^{L1}} = (Y - a^{L2}) * 1 * w^{L2} * 1 * 1  = (Y - a^{L2}) * w^{L2} \\
\frac{\partial C}{\partial w^{L2}} & = \frac{\partial C}{\partial a^{L2}} \frac{\partial a^{L2}}{\partial z^{L2}} \frac{\partial z^{L2}}{\partial w^{L2}} = (Y - a^{L2}) * 1 * a^{L1} = (Y - a^{L2}) * a^{L1} \\
\frac{\partial C}{\partial b^{L2}} & = \frac{\partial C}{\partial a^{L2}} \frac{\partial a^{L2}}{\partial z^{L2}} \frac{\partial z^{L2}}{\partial b^{L2}} = (Y - a^{L2}) * 1 * 1 = (Y - a^{L2})  \\
\end{align}

Just for the sake of clarity I will remove all of the substitution and canceling

\begin{align}
\frac{\partial C}{\partial w^{L1}} & = (Y - a^{L2}) * w^{L2} * x \\
\frac{\partial C}{\partial b^{L1}} & = (Y - a^{L2}) * w^{L2} \\
\frac{\partial C}{\partial w^{L2}} & = (Y - a^{L2}) * a^{L1} \\
\frac{\partial C}{\partial b^{L2}} & = (Y - a^{L2})  \\
\end{align}

And finally because I told you we could do this with any number of layers, here are the generic equations where $1 <= K < N$ and $N$ is our number of layers. Remember the super script of each term just says which layer we're talking about and the fractions are just the effect that a particular input (the denominator) has on the output (the numerator) of that step in the chain. 

The right 2 terms tell us what effect the weights in layer $K$ have on the output activation of layer $K$. The middle term in parenthesis gets expanded for each subsequent layer. In english the middle term tells us what effect the activation from layer $K$ has on the output activation of layer $K+1$ up to $N$ and the last term tells us what effect the output activation from layer $N$ has on the cost function. The funky $\prod$ symbol just means multiply everything together.

I like to think about the top and bottom terms canceling when they are the same. In other words dividing something by itself just gives you 1. Therefore, we can add as many terms in the middle series as we want because their numerators and denominators cancel out.

\begin{equation*}
\frac{\partial C}{\partial w^K} = \frac{\partial C}{\partial a^N} * \left(\prod_{l = K + 1}^{N} \frac{\partial a^l}{\partial z^l} * \frac{\partial z^l}{\partial a^{l-1}}\right) * \frac{\partial a^K} {\partial z^K} * \frac{\partial z^K}{\partial w^K}
\end{equation*}


# Let's Do It!


In [10]:
class Network:
    def __init__(self, *layers, cost_fn=cost):
        self.layers = layers
        self.dz_dw = []
        
    def predict(self, x):
        dz_dw = [x]
        prediction = x

        for l in self.layers:
            prediction = l.predict(prediction)
            dz_dw.append(prediction)

        self.dz_dw = dz_dw

        return prediction
    
    
    def get_gradient(self, x, y):
        # find the gradient for a single example
        prediction = self.predict(x)
        # get the gradient of the cost function
        dC_da = prediction - y

        dC_dw = []
        dC_db = []

        # loop over the layers in reverse using python's handy
        # negative list index where list[-1] is the last element,
        # list[-2] is second to last, and so on.
        for i in range(1, len(self.layers)+1):
            dC_dz = dC_da
            # always insert the weight and bias gradients in the
            # beginning of the list b/c we're going in reverse
            # that way, when we're done, the gradients are in
            # forward order as they appear in the network
            dC_dw.insert(0, np.outer(dC_dz, self.dz_dw[-(i+1)]))
            dC_db.insert(0, dC_dz)
            dC_da = self.layers[-i].weights.transpose().dot(dC_dz)

        return dC_dw, dC_db


def train(net, features, labels, epochs, learning_rate):
    for epoch in range(epochs):
        for (house, price) in zip(features, labels):
                dC_dws, dC_dbs = net.get_gradient(house, price)
                # THIS IS THE LEARNING
                # here we update all the weights and biases
                for j in range(len(net.layers)):
                    net.layers[j].weights -= dC_dws[j] * learning_rate
                    net.layers[j].biases -= dC_dbs[j].flatten() * learning_rate




num_inputs = len(house_features[0])
num_neurons_l1 = 5
num_neurons_l2 = 3

l1 = Layer(num_inputs, num_neurons_l1)
l2 = Layer(num_neurons_l1, num_neurons_l2)
l3 = Layer(num_neurons_l2, 1)

net = Network(l1, l2, l3)


test_start = randint(0, len(house_features)-10)
test_set = house_features_normalized[test_start:test_start+10]
actual_values = house_prices[test_start:test_start+10]

# The learning rate is so small because of
# the exploding gradient problem. As we backpropagate
# the gradients, we multiply by the weights in each 
# layer and sum. As you can imagine repeatedly multiplying
# and adding can grow very quickly! Therefore, to prevent
# the gradients from running off to infinity, we must scale 
# them down heavily.
learning_rate = .00005

print('\n====== BEFORE TRAINING ======')
evaluate_predictor(test_set, actual_values, net.predict)

train(net, house_features_normalized, house_prices, 200, learning_rate)

print('\n====== AFTER TRAINING ======')
evaluate_predictor(test_set, actual_values, net.predict)


====== BEFORE TRAINING ======
cost: 106.50

predicted	actual
---------------------
-0.08		13.30
-0.08		17.80
-0.08		14.00
-0.08		14.40
-0.08		13.40
-0.08		15.60
-0.08		11.80
-0.08		13.80
-0.08		15.60
-0.08		14.60

====== AFTER TRAINING ======
cost: 13.48

predicted	actual
---------------------
19.78		13.30
19.97		17.80
19.65		14.00
19.41		14.40
20.16		13.40
20.17		15.60
20.10		11.80
14.64		13.80
15.01		15.60
20.01		14.60


# A Visual of what we just made

This is the image of the network from the previous example. The weight matricies show how the dimensions line up. The rows of the matrix match the number of neurons on the output side while the columns correspond to the neurons on the input side. Now that you know what each component of the network is, a great online visualization is available [here](https://playground.tensorflow.org/#activation=tanh&batchSize=10&dataset=circle&regDataset=reg-plane&learningRate=0.03&regularizationRate=0&noise=0&networkShape=4,2&seed=0.99090&showTestData=false&discretize=false&percTrainData=50&x=true&y=true&xTimesY=false&xSquared=false&ySquared=false&cosX=false&sinX=false&cosY=false&sinY=false&collectStats=false&problem=classification&initZero=false&hideText=false) 


![net-example](img/net-example.svg)